Analisis de los tamaños de los petido y MHC, con el objetivo de definir un tamaño maximo y hacer padding

In [4]:
import pandas as pd

data = pd.read_csv('../dataset/netMHCIIpan3.2/train.csv')
#print(data)

print("TRAIN")
print(f"max peptide length {data.peptide.str.len().max()}")
print(f"min peptide length {data.peptide.str.len().min()}")
print(f"mean peptide length {data.peptide.str.len().mean()}")
print(f"max MHC length {data.mhc.str.len().max()}")
print(f"min MHC length {data.mhc.str.len().min()}")


data = pd.read_csv('../dataset/netMHCIIpan3.2/test.csv')
#print(data)

print("\nTEST")
print(f"max peptide length {data.peptide.str.len().max()}")
print(f"min peptide length {data.peptide.str.len().min()}")
print(f"max MHC length {data.mhc.str.len().max()}")
print(f"min MHC length {data.mhc.str.len().min()}")


data = pd.read_csv('../dataset/netMHCIIpan3.2/eval.csv')
#print(data)

print("\nVAL")
print(f"max peptide length {data.peptide.str.len().max()}")
print(f"min peptide length {data.peptide.str.len().min()}")
print(f"max MHC length {data.mhc.str.len().max()}")
print(f"min MHC length {data.mhc.str.len().min()}")

TRAIN
max peptide length 37
min peptide length 9
mean peptide length 15.313720551257678
max MHC length 34
min MHC length 34

TEST
max peptide length 37
min peptide length 9
max MHC length 34
min MHC length 34

VAL
max peptide length 37
min peptide length 9
max MHC length 34
min MHC length 34


In [1]:
import pandas as pd

data = pd.read_csv('dataset/hlab/hlab_train.csv')
#print(data)

print("TRAIN")
print(f"max peptide length {data.peptide.str.len().max()}")
print(f"min peptide length {data.peptide.str.len().min()}")
print(f"mean peptide length {data.peptide.str.len().mean()}")
print(f"max MHC length {data.mhc.str.len().max()}")
print(f"min MHC length {data.mhc.str.len().min()}")


data = pd.read_csv('dataset/hlab/hlab_test.csv')
#print(data)

print("\nTEST")
print(f"max peptide length {data.peptide.str.len().max()}")
print(f"min peptide length {data.peptide.str.len().min()}")
print(f"max MHC length {data.mhc.str.len().max()}")
print(f"min MHC length {data.mhc.str.len().min()}")


data = pd.read_csv('dataset/hlab/hlab_val.csv')
#print(data)

print("\nVAL")
print(f"max peptide length {data.peptide.str.len().max()}")
print(f"min peptide length {data.peptide.str.len().min()}")
print(f"max MHC length {data.mhc.str.len().max()}")
print(f"min MHC length {data.mhc.str.len().min()}")

TRAIN
max peptide length 14
min peptide length 8
mean peptide length 9.462156250521781
max MHC length 34
min MHC length 34

TEST
max peptide length 14
min peptide length 8
max MHC length 34
min MHC length 34

VAL
max peptide length 14
min peptide length 8
max MHC length 34
min MHC length 34


# Predictions netMHCpan4.1

In [2]:
import pandas as pd
import os

data = pd.read_csv('dataset/hlab/hlab_test.csv')
hla_list = data.HLA.unique()
data = data.sort_values(by=['HLA'], ascending=True)
data = data.set_index(['HLA'])
#print( data)
#print(data.loc["HLA-A*01:01"]["mhc"])
#print(data.loc[hla]["peptide"])

for hla in hla_list:
    # peptides
    hla_peptide = data.loc[hla]["peptide"]
    print("HLA:", hla)    

    # save peptides
    #hla_peptide.to_csv("netMHCpan-4.1/test/tmp_peptides.pep", index=False, header=False)
    hla_peptide.to_csv("tmp_peptides.pep", index=False, header=False)

    #cmd = "cd netMHCpan-4.1/test; tcsh ../netMHCpan.sh -a " + hla.replace("*", "") + " -p tmp_peptides.pep -hlapseudo /home/vicente/projects/pmhc/dataset/hlab/MHC_pseudo.dat > tmp_results_netMHCpan4.1"
    cmd = "tcsh netMHCpan-4.1/netMHCpan.sh -a " + hla.replace("*", "") + " -p tmp_peptides.pep -hlapseudo /home/vicente/projects/pmhc/dataset/hlab/MHC_pseudo.dat -xls -xlsfile tmp_results.xls"
    print(cmd)
    os.system(cmd)
    break

HLA: HLA-A*01:01
tcsh netMHCpan-4.1/netMHCpan.sh -a HLA-A01:01 -p tmp_peptides.pep -hlapseudo /home/vicente/projects/pmhc/dataset/hlab/MHC_pseudo.dat -xls -xlsfile tmp_results.xls
testing
# /home/vicente/projects/pmhc/netMHCpan-4.1/Linux_x86_64/bin/netMHCpan -a HLA-A01:01 -p tmp_peptides.pep -hlapseudo /home/vicente/projects/pmhc/dataset/hlab/MHC_pseudo.dat -xls -xlsfile tmp_results.xls
# Fri Jun  9 09:21:39 2023
# User: vicente
# PWD : /home/vicente/projects/pmhc
# Host: Linux vicente-desktop 5.15.0-73-generic x86_64
# -a       HLA-A01:01           MHC allele
# -p       1                    Use peptide input
# -hlapseudo /home/vicente/projects/pmhc/dataset/hlab/MHC_pseudo.dat File with MHC pseudo sequences
# -xls     1                    Save output to xls file
# -xlsfile tmp_results.xls      Filename for xls dump
# Command line parameters set to:
#	[-rdir filename]     /home/vicente/projects/pmhc/netMHCpan-4.1/Linux_x86_64 Home directory for NetMHpan
#	[-syn filename]      /home/vice

paste: write error: Broken pipe
paste: write error
paste: write error: Broken pipe
paste: write error
